In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"""
# Community Helper Guidebook — Low-Resource Helpbook in India

**What this IS:**
A multimodal, offline-first assistive tool that converts spoken notes and
images from community health workers into safe, simplified, multilingual
guidance packs — grounded in real Indian healthcare regulations.

**What this is NOT:** A chatbot, diagnostic engine, or treatment recommender.

---

### Full Pipeline
```
  Audio (upload or record live)     Text input         Image upload
          ↓                               ↓                   ↓
   Whisper STT                  [skip confirmation]     SigLIP encoding
   (transcribe mode)
          ↓
    Confirmation Layer
   (helper reviews & corrects transcript — skipped for text input)
          ↓
   IndicTrans2  →  local language to English
          ↓
   ★ RAG Retrieval ★
   (FAISS + sentence-transformers over real Indian MoHFW/NHM documents)
          ↓  ←──────────────────────────── image context
   MedGemma ReAct Agent  (grounded in Indian healthcare regulations)
     ├── get_first_aid_steps(situation)
     ├── check_urgency(symptoms)
     ├── lookup_nearest_facility(region)
     └── generate_referral_note(case_summary)
          ↓
   Safety Guardrails
          ↓
   IndicTrans2  →  English to local language
          ↓
 Helper Guidance Pack
```

---
> This system never diagnoses, prescribes, or recommends treatments.
> Every output instructs escalation to a qualified healthcare professional.
> Designed for community helpers — NOT for patients directly.
"""

In [48]:
# Install Dependencies
import subprocess, sys

def pip(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

pip("transformers>=4.40.0")
pip("accelerate>=0.27.0")
pip("bitsandbytes>=0.43.0")
pip("openai-whisper")
pip("pydub")
pip("ffmpeg-python")
pip("Pillow")
pip("gradio>=4.0.0")
pip("sentencepiece")
pip("protobuf")
pip("faiss-cpu")
pip("sentence-transformers")
pip("PyMuPDF")          # fitz — PDF text extraction
pip("deep-translator")

print("All dependencies installed.")

All dependencies installed.


In [3]:
# Imports and Configs

import os
import re
import json
import warnings
import tempfile
import urllib.request
from datetime import datetime
from pathlib import Path

import torch
import numpy as np
import faiss
import fitz                          # PyMuPDF
import whisper
from PIL import Image, ImageDraw
from pydub import AudioSegment
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoProcessor,
    AutoModel,
    BitsAndBytesConfig,
)

warnings.filterwarnings("ignore")

# ── Device ────────────────────────────────────────────────────────
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE    = torch.bfloat16 if torch.cuda.is_available() else torch.float32
USE_4BIT = torch.cuda.is_available()

# ── Model IDs ─────────────────────────────────────────────────────
MEDGEMMA_ID   = "google/medgemma-4b-it"
MEDSIG_ID     = "google/siglip-so400m-patch14-384"
WHISPER_SIZE  = "medium"
EMBEDDER_ID   = "sentence-transformers/all-MiniLM-L6-v2"

# ── Supported languages ───────────────────────────────────────────
SUPPORTED_LANGUAGES = {
    "Hindi"   : "hi",
    "Tamil"   : "ta",
    "Bengali" : "bn",
    "Telugu"  : "te",
    "Marathi" : "mr",
    "Gujarati": "gu",
    "Kannada" : "kn",
    "English" : "en",
}

# ── Safety disclaimer ─────────────────────────────────────────────
DISCLAIMER = (
    "\n\n─────────────────────────────────────────────\n"
    " IMPORTANT NOTICE\n"
    "This is assistive guidance for community health workers only.\n"
    "It does NOT replace clinical advice or medical examination.\n"
    "Always escalate to a qualified healthcare professional.\n"
    "Emergency Ambulance: Call 108 (free, 24/7)\n"
    "─────────────────────────────────────────────"
)

print(f"Config ready")
print(f"   Device   : {DEVICE.upper()}")
print(f"   Whisper  : {WHISPER_SIZE}")
print(f"   MedGemma : {MEDGEMMA_ID}")
print(f"   Embedder : {EMBEDDER_ID}")

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Config ready
   Device   : CUDA
   Whisper  : medium
   MedGemma : google/medgemma-4b-it
   Embedder : sentence-transformers/all-MiniLM-L6-v2


In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

secrets  = UserSecretsClient()
hf_token = secrets.get_secret("HF_TOKEN")
login(token=hf_token)
print("HuggingFace login successful.")

HuggingFace login successful.


In [5]:
# Load MedGemma 4B-IT (4-bit quantized)
print("Loading MedGemma 4B — may take 3–5 minutes on first run...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit              = True,
    bnb_4bit_quant_type       = "nf4",
    bnb_4bit_compute_dtype    = DTYPE,
    bnb_4bit_use_double_quant = True,
) if USE_4BIT else None

tokenizer = AutoTokenizer.from_pretrained(
    MEDGEMMA_ID, trust_remote_code=True
)

medgemma = AutoModelForCausalLM.from_pretrained(
    MEDGEMMA_ID,
    quantization_config = bnb_config,
    device_map          = "auto" if torch.cuda.is_available() else None,
    torch_dtype         = DTYPE,
    trust_remote_code   = True,
)
medgemma.eval()
print("MedGemma 4B loaded.")

Loading MedGemma 4B — may take 3–5 minutes on first run...


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

MedGemma 4B loaded.


In [6]:
# Load SigLIP (MedSigLIP proxy)
# NOTE: SigLIP is used as a capable zero-shot proxy for MedSigLIP.
# In production: replace with a fine-tuned MedSigLIP checkpoint

print(" Loading SigLIP image encoder (MedSigLIP proxy)...")

siglip_processor = AutoProcessor.from_pretrained(MEDSIG_ID)
siglip_model     = AutoModel.from_pretrained(MEDSIG_ID, torch_dtype=DTYPE)
siglip_model     = siglip_model.to(DEVICE).eval()

print("SigLIP loaded.")

 Loading SigLIP image encoder (MedSigLIP proxy)...


preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

The image processor of type `SiglipImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/888 [00:00<?, ?it/s]

SigLIP loaded.


In [7]:
# Load Whisper STT
print(f"Loading Whisper ({WHISPER_SIZE})...")
whisper_model = whisper.load_model(WHISPER_SIZE, device=DEVICE)
print("Whisper loaded.")

Loading Whisper (medium)...


100%|█████████████████████████████████████| 1.42G/1.42G [00:19<00:00, 80.4MiB/s]


Whisper loaded.


In [8]:
# Load Sentence Embedder (for RAG)
print(f"Loading sentence embedder ({EMBEDDER_ID})...")
embedder = SentenceTransformer(EMBEDDER_ID)
print("Sentence embedder loaded.")

Loading sentence embedder (sentence-transformers/all-MiniLM-L6-v2)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence embedder loaded.


In [36]:
# RAG — Indian Healthcare Document Knowledge Base
# RAG KNOWLEDGE BASE                                                                                                       
# Sources:                                                    
# 1. National Health Policy 2017 (MoHFW)                     
# 2. IPHS Guidelines for CHC/PHC 2022 (NHM)                  
# 3. NHM Community Health Worker Guidelines                   
# 4. Key excerpts: ASHA Training Module + AB PM-JAY           
#     (embedded directly — guaranteed availability)            
                                                           
# Note: In production, LangChain + ChromaDB can replace       
# this FAISS implementation for easier document management,   
# persistent storage, and metadata filtering.                 


# Real document URLs (public government sources)

DOCUMENT_URLS = {
    "national_health_policy_2017": (
        "https://www.mohfw.gov.in/sites/default/files/9147562941489753121.pdf"
    ),
    "iphs_guidelines_chc_phc": (
        "https://nhm.gov.in/images/pdf/guidelines/iphs/iphs-revised-guidlines-2012/"
        "primay-health-centres.pdf"
    ),
    "nhm_community_health": (
        "https://nhm.gov.in/images/pdf/communitisation/resource-material/"
        "CHW-Guidelines.pdf"
    ),
}

# ── Key excerpts embedded directly ───────────────────────────────
# These cover ASHA protocols and Ayushman Bharat — included as text
# to guarantee availability regardless of PDF accessibility.

EMBEDDED_EXCERPTS = [
    # ── ASHA Worker Protocols (MoHFW Training Module) ─────────────
    {
        "source" : "ASHA Training Module — MoHFW",
        "text"   : (
            "ASHA (Accredited Social Health Activist) is a trained female community "
            "health activist. She is the first port of call for any health related "
            "demands of deprived sections of the population, especially women and "
            "children who find it difficult to access health services. "
            "ASHA is expected to be a resident of the village, married, widowed or "
            "divorced, preferably in the age group of 25 to 45 years, with formal "
            "education up to 8th class. "
            "ASHA acts as an interface between the community and the public health system."
        ),
    },
    {
        "source" : "ASHA Training Module — MoHFW",
        "text"   : (
            "ASHA's role is not to diagnose or treat illness. Her role is to counsel "
            "community members on health, nutrition, and hygiene. She mobilises the "
            "community for health services, facilitates access to improved healthcare "
            "at the community level, and acts as the first contact for the community "
            "with the health system. ASHA is a link worker, not a healthcare provider. "
            "She should refer serious cases to the Sub-Centre, PHC, or CHC immediately."
        ),
    },
    {
        "source" : "ASHA Training Module — MoHFW",
        "text"   : (
            "Danger signs requiring immediate referral by ASHA: "
            "High fever with convulsions, unconsciousness, difficulty breathing, "
            "severe bleeding, snake bite, poisoning, chest pain, prolonged labour, "
            "newborn not breathing, severe malnutrition with complications. "
            "In all such cases ASHA must call 108 ambulance and accompany patient "
            "to the nearest PHC or District Hospital."
        ),
    },
    {
        "source" : "ASHA Training Module — MoHFW",
        "text"   : (
            "First aid role of ASHA: ASHA is trained only in basic first aid. "
            "This includes keeping the patient calm, placing the patient in a safe "
            "position, preventing further harm, and arranging for immediate referral. "
            "ASHA must not attempt to diagnose illness, prescribe medicines, or "
            "perform clinical procedures. If in doubt, always refer to a health worker."
        ),
    },

    # ── PHC/CHC Referral Protocols (NHM IPHS Guidelines) ──────────
    {
        "source" : "IPHS Guidelines for PHC — NHM 2022",
        "text"   : (
            "Primary Health Centre (PHC) is the first contact point between village "
            "community and the Medical Officer. The PHC covers a population of 30,000 "
            "in plain areas and 20,000 in hilly or tribal areas. "
            "PHC provides curative, preventive, promotive and rehabilitative primary "
            "healthcare services to the rural population. "
            "Referral from PHC to CHC or District Hospital is mandatory for cases "
            "requiring specialist care, surgery, or intensive management."
        ),
    },
    {
        "source" : "IPHS Guidelines for CHC — NHM 2022",
        "text"   : (
            "Community Health Centre (CHC) serves as a referral centre for PHCs. "
            "CHC covers a population of 1,20,000 in plain areas. "
            "It provides specialist services in Medicine, Surgery, Obstetrics, "
            "Gynaecology, and Paediatrics. "
            "Cases beyond CHC capacity are referred to District Hospital or "
            "Medical College Hospital. "
            "Emergency services at CHC are available 24 hours, 7 days a week."
        ),
    },
    {
        "source" : "IPHS Guidelines — NHM 2022",
        "text"   : (
            "Referral protocol: Community health worker identifies case → "
            "Provides immediate first aid → Calls 108 ambulance if urgent → "
            "Refers to Sub-Centre or PHC with written referral note → "
            "PHC assesses and refers to CHC if needed → "
            "CHC refers to District Hospital for specialist care. "
            "All referrals must be accompanied by a written note describing "
            "the patient's condition and actions taken."
        ),
    },

    # ── National Health Policy 2017 ────────────────────────────────
    {
        "source" : "National Health Policy 2017 — MoHFW",
        "text"   : (
            "The National Health Policy 2017 aims to achieve the highest possible "
            "level of good health and well-being through a preventive and promotive "
            "healthcare orientation in all developmental policies. "
            "It advocates reaching healthcare to the doorstep of citizens in rural "
            "and underserved areas through community health workers. "
            "The policy emphasises that community health workers should be empowered "
            "with information and tools — not burdened with clinical responsibilities "
            "beyond their training."
        ),
    },
    {
        "source" : "National Health Policy 2017 — MoHFW",
        "text"   : (
            "Digital health is recognised in the National Health Policy 2017 as a "
            "key enabler for health system strengthening. "
            "Technology-enabled tools for community health workers should support "
            "record-keeping, referral, and communication with higher health facilities. "
            "Such tools must be accessible in local languages and usable by workers "
            "with limited literacy. Privacy and confidentiality of patient data "
            "must be maintained at all times."
        ),
    },

    # ── Ayushman Bharat PM-JAY ─────────────────────────────────────
    {
        "source" : "Ayushman Bharat PM-JAY — NHA Operational Guidelines",
        "text"   : (
            "Ayushman Bharat Pradhan Mantri Jan Arogya Yojana (PM-JAY) provides "
            "health cover of Rs. 5 lakhs per family per year for secondary and "
            "tertiary hospitalisation. "
            "Beneficiaries include families identified as poor and vulnerable as per "
            "SECC database. The scheme covers over 10.74 crore poor and vulnerable "
            "families — approximately 50 crore beneficiaries. "
            "All public hospitals and empanelled private hospitals are covered. "
            "Community health workers should inform eligible families about PM-JAY "
            "benefits and assist them in accessing scheme cards."
        ),
    },
    {
        "source" : "Ayushman Bharat PM-JAY — NHA Operational Guidelines",
        "text"   : (
            "Under Ayushman Bharat, Health and Wellness Centres (HWCs) are the "
            "first level of care for the community. "
            "HWCs provide comprehensive primary healthcare including maternal health, "
            "child health, communicable and non-communicable diseases, and basic "
            "emergency services. "
            "Community health workers must refer patients to the nearest HWC or PHC "
            "as the first point of formal medical contact. "
            "Emergency referrals should always use the 108 ambulance service."
        ),
    },

    # ── NHM Community Health Worker Guidelines ─────────────────────
    {
        "source" : "NHM Community Health Worker Guidelines",
        "text"   : (
            "Community health workers must follow these core principles: "
            "1. Do no harm — never attempt procedures beyond your training. "
            "2. Refer early — do not delay referral for serious conditions. "
            "3. Communicate clearly — use simple language the family understands. "
            "4. Document everything — record all observations and actions taken. "
            "5. Maintain confidentiality — do not share patient information publicly. "
            "6. Follow up — check on referred patients to ensure they received care."
        ),
    },
    {
        "source" : "NHM Community Health Worker Guidelines",
        "text"   : (
            "Community health workers should NEVER: "
            "Diagnose any illness or condition. "
            "Prescribe or dispense medications beyond the approved drug kit. "
            "Perform injections unless specifically trained and authorised. "
            "Conduct deliveries without trained supervision. "
            "Give dietary supplements without proper assessment. "
            "Make promises about recovery or prognosis. "
            "Delay referral to attempt home treatment for serious conditions."
        ),
    },
]


def extract_pdf_text(pdf_path: str, max_chars: int = 50000) -> str:
    """Extracts text from a PDF file using PyMuPDF (fitz)."""
    doc  = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
        if len(text) > max_chars:
            break
    return text[:max_chars]


def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100) -> list[str]:
    """
    Splits text into overlapping chunks for embedding.
    Overlap ensures context is preserved across chunk boundaries.
    """
    words  = text.split()
    chunks = []
    start  = 0
    while start < len(words):
        end   = min(start + chunk_size, len(words))
        chunk = " ".join(words[start:end])
        if len(chunk.strip()) > 50:   # skip very short chunks
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks


def download_pdf(url: str, name: str) -> str | None:
    """Downloads a PDF to /tmp. Returns path or None if failed."""
    path = f"/tmp/{name}.pdf"
    if os.path.exists(path):
        print(f"   ✓ {name} already cached.")
        return path
    try:
        print(f"   ↳ Downloading {name}...")
        urllib.request.urlretrieve(url, path)
        return path
    except Exception as e:
        print(f"   ❌ TRANSLATION FAILED: {e}")
        return text, "passthrough (translation failed)"


def build_rag_index() -> tuple[faiss.Index, list[dict]]:
    """
    Builds FAISS vector index from:
    1. Real Indian government PDFs (downloaded at runtime)
    2. Embedded excerpts (ASHA protocols, PM-JAY, NHM guidelines)

    Returns (faiss_index, document_store).
    document_store: list of {text, source} dicts aligned with FAISS index.

    NOTE: In production, LangChain + ChromaDB can replace this FAISS
    implementation for easier document management, persistent vector
    storage, metadata filtering, and hybrid search.
    """
    all_chunks = []

    # ── Step 1: Download and extract real PDFs ─────────────────────
    print("\nBuilding RAG knowledge base from Indian healthcare documents...")
    for name, url in DOCUMENT_URLS.items():
        pdf_path = download_pdf(url, name)
        if pdf_path:
            try:
                raw_text = extract_pdf_text(pdf_path)
                chunks   = chunk_text(raw_text)
                for chunk in chunks:
                    all_chunks.append({"text": chunk, "source": name})
                print(f"   ✓ {name}: {len(chunks)} chunks extracted.")
            except Exception as e:
                print(f" Could not extract text from {name}: {e}")

    # ── Step 2: Add embedded excerpts ─────────────────────────────
    print(f"   ↳ Adding {len(EMBEDDED_EXCERPTS)} embedded regulatory excerpts...")
    for excerpt in EMBEDDED_EXCERPTS:
        chunks = chunk_text(excerpt["text"])
        for chunk in chunks:
            all_chunks.append({"text": chunk, "source": excerpt["source"]})

    print(f"\n   Total chunks in knowledge base: {len(all_chunks)}")

    # ── Step 3: Embed all chunks ───────────────────────────────────
    print("   ↳ Embedding chunks...")
    texts      = [c["text"] for c in all_chunks]
    embeddings = embedder.encode(texts, show_progress_bar=True, batch_size=64)
    embeddings = np.array(embeddings, dtype=np.float32)

    # ── Step 4: Build FAISS index ──────────────────────────────────
    dim   = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    print(f"RAG index built: {index.ntotal} vectors indexed.")
    return index, all_chunks


def retrieve_context(
    query      : str,
    index      : faiss.Index,
    doc_store  : list[dict],
    top_k      : int = 4,
) -> str:
    """
    Retrieves top-k most relevant chunks from the knowledge base
    for a given query. Returns formatted context string for MedGemma.
    """
    query_vec = embedder.encode([query], show_progress_bar=False)
    query_vec = np.array(query_vec, dtype=np.float32)

    distances, indices = index.search(query_vec, top_k)

    context_parts = []
    for rank, idx in enumerate(indices[0]):
        if idx < len(doc_store):
            chunk  = doc_store[idx]
            source = chunk["source"]
            text   = chunk["text"]
            context_parts.append(
                f"[Source {rank+1}: {source}]\n{text}"
            )

    return "\n\n".join(context_parts)


# ── Build the index at startup ─────────────────────────────────────
rag_index, rag_doc_store = build_rag_index()

print("\n RAG module ready.")


Building RAG knowledge base from Indian healthcare documents...
   ✓ national_health_policy_2017 already cached.
   ✓ national_health_policy_2017: 18 chunks extracted.
   ✓ iphs_guidelines_chc_phc already cached.
   ✓ iphs_guidelines_chc_phc: 18 chunks extracted.
   ✓ nhm_community_health already cached.
   ✓ nhm_community_health: 1 chunks extracted.
   ↳ Adding 13 embedded regulatory excerpts...

   Total chunks in knowledge base: 50
   ↳ Embedding chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

RAG index built: 50 vectors indexed.

 RAG module ready.


In [10]:
# Module A — Whisper STT + pydub Audio Conversion

SUPPORTED_AUDIO_FORMATS = [".wav", ".mp3", ".m4a", ".ogg", ".flac", ".webm"]


def convert_audio(input_path: str) -> str:
    """
    Converts any audio format to 16kHz mono WAV using pydub + ffmpeg.
    Whisper performs best on 16kHz mono WAV.
    Returns path to converted temp file.
    """
    ext = Path(input_path).suffix.lower()
    if ext not in SUPPORTED_AUDIO_FORMATS:
        raise ValueError(
            f"Unsupported format: {ext}. Supported: {SUPPORTED_AUDIO_FORMATS}"
        )

    audio = AudioSegment.from_file(input_path)
    audio = audio.set_channels(1).set_frame_rate(16000)

    out_path = tempfile.mktemp(suffix=".wav")
    audio.export(out_path, format="wav")
    return out_path


def transcribe_audio(audio_path: str, language: str = "hi") -> tuple[str, str]:
    """
    Transcribes audio using Whisper in TRANSCRIBE mode.
    Output stays in source language — IndicTrans2 handles translation.
    Returns (transcript, detected_language).
    """
    converted = convert_audio(audio_path)

    result = whisper_model.transcribe(
        converted,
        language = language,
        task     = "transcribe",
        fp16     = torch.cuda.is_available(),
    )

    transcript        = result["text"].strip()
    detected_language = result.get("language", language)

    try:
        os.remove(converted)
    except Exception:
        pass

    return transcript, detected_language


print("STT module ready (Whisper + pydub).")

STT module ready (Whisper + pydub).


In [11]:
# Module B — Confirmation Layer

def prepare_confirmation(
    transcript : str,
    input_mode : str,
    detected   : str = "",
) -> dict:
    """
    Returns confirmation metadata.
    - Text input  → skip = True
    - Audio input → skip = False (helper reviews editable transcript)
    """
    if input_mode == "text":
        return {
            "skip"    : True,
            "message" : "Text input — confirmation step skipped.",
        }
    return {
        "skip"    : False,
        "message" : (
            f"🎙️ Whisper transcribed the following "
            f"(detected language: {detected}).\n"
            "Please review and correct any errors before generating guidance:"
        ),
    }


print("Confirmation layer ready.")

Confirmation layer ready.


In [49]:
# Module C — IndicTrans2 Translation (mock, swap-in ready)

from deep_translator import GoogleTranslator

# ╔═══════════════════════════════════════════════════════════════╗
# ║  PRODUCTION SWAP-IN                                          ║
# ║  Replace GoogleTranslator calls with IndicTrans2:            ║
# ║                                                              ║
# ║  from IndicTrans2 import Translator                          ║
# ║  t = Translator(src_lang=src_code, tgt_lang="en")            ║
# ║  return t.translate(text), "IndicTrans2"                     ║
# ╚═══════════════════════════════════════════════════════════════╝

def translate_to_english(text: str, source_lang: str = "Hindi") -> tuple[str, str]:
    """
    Translates local-language text to English using Google Translate.
    Production swap-in: replace with IndicTrans2 for higher accuracy
    on Indian languages and offline capability.
    """
    if source_lang == "English":
        return text, "passthrough"

    lang_code = SUPPORTED_LANGUAGES.get(source_lang, "hi")

    try:
        # Chunk to stay under Google Translate 5000 char limit
        chunks     = [text[i:i+4500] for i in range(0, len(text), 4500)]
        translated = " ".join(
            GoogleTranslator(source=lang_code, target="en").translate(chunk)
            for chunk in chunks
        )
        return translated, "GoogleTranslate (demo)"

    except Exception as e:
        print(f" Traslation to English failed: {e} — using original text")
        return text, "passthrough (failed)"


def translate_from_english(text: str, target_lang: str = "Hindi") -> tuple[str, str]:
    """
    Translates English guidance back to local language using Google Translate.
    Production swap-in: replace with IndicTrans2 for higher accuracy
    on Indian languages and offline capability.
    """
    if target_lang == "English":
        return text, "passthrough"

    lang_code = SUPPORTED_LANGUAGES.get(target_lang, "hi")

    try:
        # Chunk to stay under Google Translate 5000 char limit
        chunks     = [text[i:i+4500] for i in range(0, len(text), 4500)]
        translated = " ".join(
            GoogleTranslator(source="en", target=lang_code).translate(chunk)
            for chunk in chunks
        )
        return translated, "GoogleTranslate (demo)"

    except Exception as e:
        print(f"   Translation from English failed: {e} — returning English")
        return text, "passthrough (failed)"


print("Translation module ready (Google Translate — IndicTrans2 swap-in ready).")



Translation module ready (Google Translate — IndicTrans2 swap-in ready).


In [13]:
# Module D — SigLIP Image Encoding

MEDICAL_IMAGE_LABELS = [
    "chest X-ray",
    "skin lesion or open wound",
    "eye or retina photograph",
    "handwritten medical prescription",
    "ultrasound scan image",
    "ECG or EKG tracing",
    "burn injury on skin",
    "skin rash or dermatological condition",
    "fracture or bone X-ray",
    "general medical document or lab report",
]


def encode_image(image: Image.Image) -> dict:
    """
    Zero-shot medical image classification using SigLIP.
    Returns image type, confidence, and a context string for MedGemma.
    Does NOT provide diagnosis — supporting context only.
    """
    inputs = siglip_processor(
        text           = MEDICAL_IMAGE_LABELS,
        images         = image,
        return_tensors = "pt",
        padding        = True,
    ).to(DEVICE)

    with torch.no_grad():
        outputs = siglip_model(**inputs)
        probs   = torch.softmax(
            outputs.logits_per_image[0], dim=-1
        ).float().cpu().numpy()

    top_idx   = int(np.argmax(probs))
    top_label = MEDICAL_IMAGE_LABELS[top_idx]
    top_score = float(probs[top_idx])

    return {
        "image_type"      : top_label,
        "confidence"      : round(top_score, 3),
        "all_labels"      : {
            label: round(float(p), 3)
            for label, p in zip(MEDICAL_IMAGE_LABELS, probs)
        },
        "context_for_llm" : (
            f"The community helper has uploaded an image. "
            f"It appears to show: {top_label} "
            f"(confidence: {top_score:.0%}). "
            f"Use this as supporting context only. "
            f"Do NOT provide a diagnosis based on this image."
        ),
    }


print("Image encoding module ready (SigLIP / MedSigLIP proxy).")


Image encoding module ready (SigLIP / MedSigLIP proxy).


In [44]:
# Module E — Agentic Tool Definitions


# TOOL 1: get_first_aid_steps
# MedGemma generates bare-hands first aid steps.
# Strict prompt boundary: no medication, no clinical procedures.
# First aid definition: immediate care before professional help arrives.


FIRST_AID_SYSTEM_PROMPT = """You are a first aid assistant for community health workers in rural India.

Generate ONLY first aid steps for the given situation.
Steps must be bare-hands only — no medication, no equipment, no clinical procedures.

Respond in this EXACT format and nothing else. Do not repeat. Stop after the last step:

DO THIS NOW:
1. [one specific step for this situation]
2. [one specific step for this situation]
3. [one specific step for this situation]

DO NOT DO:
1. [one thing to avoid for this situation]
2. [one thing to avoid for this situation]"""

def call_medgemma_prompt(FIRST_AID_SYSTEM_PROMPT, max_new_tokens=500):
    inputs = tokenizer(
        FIRST_AID_SYSTEM_PROMPT,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    input_ids = inputs["input_ids"].to(medgemma.device)
    attention_mask = inputs.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(medgemma.device)

    with torch.no_grad():
        output_ids = medgemma.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )

    return tokenizer.decode(
        output_ids[0][input_ids.shape[-1]:],
        skip_special_tokens=True
    )

def get_first_aid_steps(situation: str) -> dict:
    messages = [
        {"role": "system", "content": FIRST_AID_SYSTEM_PROMPT},
        {"role": "user",   "content": f"Situation: {situation}"},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize              = True,
        add_generation_prompt = True,
        return_tensors        = "pt",
    )

    if hasattr(input_ids, "input_ids"):
        input_ids = input_ids.input_ids
    input_ids = input_ids.to(DEVICE)

    with torch.no_grad():
        output_ids = medgemma.generate(
            input_ids,
            max_new_tokens = 150,        # strict limit — steps only
            temperature    = 0.1,        # low temp — no creativity needed
            do_sample      = True,
            pad_token_id   = tokenizer.eos_token_id,
            eos_token_id   = tokenizer.eos_token_id,
        )

    new_tokens = output_ids[0][input_ids.shape[-1]:]
    steps      = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    # Keep only the first aid block — stop at any repetition
    lines        = steps.split("\n")
    clean_lines  = []
    seen         = set()
    for line in lines:
        line_stripped = line.strip()
        if line_stripped and line_stripped not in seen:
            seen.add(line_stripped)
            clean_lines.append(line)
        elif line_stripped in seen and line_stripped.startswith("DO"):
            break  # stop at repeated section header

    steps = "\n".join(clean_lines).strip()

    return {
        "situation" : situation,
        "first_aid" : steps,
        "boundary"  : "Bare-hands only. No medication. No clinical procedures.",
    }


# TOOL 2: check_urgency
# Rule-based keyword scoring. Returns spoken-language phrase only.
# Never uses clinical labels like URGENT/CRITICAL with helpers.


URGENCY_KEYWORDS = {
    "immediate": [
        "chest pain", "difficulty breathing", "not breathing",
        "unconscious", "severe bleeding", "seizure", "fits",
        "convulsion", "snake bite", "poisoning", "stroke",
        "paralysis", "face drooping", "newborn", "premature",
        "labour", "childbirth", "not responding", "stopped breathing",
    ],
    "soon": [
        "high fever", "fever three days", "fever 3 days",
        "burn", "blistering", "vomiting", "dehydration",
        "not eating", "weakness", "fracture", "broken",
        "swelling", "wound", "injury", "dizziness",
        "rash", "not drinking",
    ],
    "routine": [
        "mild fever", "cold", "cough", "minor rash",
        "minor pain", "headache", "fatigue", "tiredness",
        "runny nose",
    ],
}

URGENCY_PHRASES = {
    "immediate": (
        "This person needs to reach a doctor or hospital RIGHT NOW. "
        "Please call 108 immediately — do not wait."
    ),
    "soon": (
        "This person should be seen by a health worker today. "
        "Please take them to the nearest health centre as soon as possible."
    ),
    "routine": (
        "This does not seem like an emergency, but the person should still "
        "visit a health worker within the next day or two."
    ),
}


def check_urgency(symptoms: str) -> dict:
    """
    Tool 2: Keyword-based urgency scoring.
    Returns spoken-language phrase — never clinical labels.
    """
    text  = symptoms.lower()
    score = {"immediate": 0, "soon": 0, "routine": 0}

    for level, keywords in URGENCY_KEYWORDS.items():
        for kw in keywords:
            if kw in text:
                score[level] += 1

    if score["immediate"] > 0:
        level = "immediate"
    elif score["soon"] > 0:
        level = "soon"
    else:
        level = "routine"

    return {
        "urgency_level"      : level,
        "urgency_phrase"     : URGENCY_PHRASES[level],
        "triggered_keywords" : [
            kw for kw in URGENCY_KEYWORDS[level] if kw in text
        ],
    }



# TOOL 3: lookup_nearest_facility
# Hardcoded India PHC/CHC database by state.
# Production: connect to NHA facility registry API or
# offline GPS-based database.

PHC_DATABASE = {
    "maharashtra"    : ["Pune District Hospital", "Nashik Civil Hospital", "PHC Sangamner"],
    "rajasthan"      : ["SMS Hospital Jaipur", "CHC Barmer", "PHC Sirohi"],
    "bihar"          : ["PMCH Patna", "Sadar Hospital Gaya", "PHC Nawada"],
    "uttar pradesh"  : ["KGMU Lucknow", "District Hospital Varanasi", "CHC Ballia"],
    "madhya pradesh" : ["Hamidia Hospital Bhopal", "MY Hospital Indore", "PHC Sehore"],
    "west bengal"    : ["SSKM Kolkata", "District Hospital Malda", "PHC Murshidabad"],
    "tamil nadu"     : ["GGH Chennai", "Coimbatore Medical College", "PHC Madurai"],
    "gujarat"        : ["Civil Hospital Ahmedabad", "New Civil Hospital Surat", "PHC Anand"],
    "karnataka"      : ["Victoria Hospital Bangalore", "District Hospital Hubli", "PHC Gulbarga"],
    "andhra pradesh" : ["GGH Vijayawada", "RIMS Kadapa", "PHC Nellore"],
    "telangana"      : ["Osmania GH Hyderabad", "District Hospital Warangal", "PHC Nizamabad"],
    "kerala"         : ["GMC Thiruvananthapuram", "District Hospital Kozhikode", "PHC Thrissur"],
    "punjab"         : ["PGIMER Chandigarh", "Civil Hospital Ludhiana", "PHC Amritsar"],
    "haryana"        : ["PGIMS Rohtak", "Civil Hospital Faridabad", "PHC Hisar"],
    "odisha"         : ["SCB Medical College Cuttack", "District Hospital Bhubaneswar", "PHC Puri"],
    "jharkhand"      : ["RIMS Ranchi", "Sadar Hospital Dhanbad", "PHC Hazaribagh"],
    "chhattisgarh"   : ["AIIMS Raipur", "District Hospital Bilaspur", "PHC Durg"],
    "assam"          : ["GMCH Guwahati", "District Hospital Dibrugarh", "PHC Jorhat"],
    "default"        : [
        "Nearest Primary Health Centre (PHC)",
        "Community Health Centre (CHC)",
        "Health and Wellness Centre (HWC)",
        "Call 108 — free emergency ambulance, available 24/7",
    ],
}


def lookup_nearest_facility(region: str) -> dict:
    """
    Tool 3: Returns nearest healthcare facilities for a given Indian region.
    Includes Ayushman Bharat eligibility reminder.
    """
    region_lower = region.lower().strip()
    facilities   = None

    for key, value in PHC_DATABASE.items():
        if key in region_lower or region_lower in key:
            facilities = value
            break

    if not facilities:
        facilities = PHC_DATABASE["default"]

    return {
        "region"          : region,
        "facilities"      : facilities,
        "emergency"       : "108 — National Ambulance Service (free, 24/7)",
        "scheme_reminder" : (
            "If the family is eligible for Ayushman Bharat PM-JAY, "
            "treatment at empanelled hospitals is free up to ₹5 lakhs per year."
        ),
    }



# TOOL 4: generate_referral_note
# Structured handoff note for the receiving doctor/health worker.
# Ensures professional context from CHW is preserved.


def generate_referral_note(case_summary: str) -> dict:
    """
    Tool 4: Generates a structured doctor handoff note.
    Includes timestamp, CHW observations, and disclaimer.
    """
    timestamp = datetime.now().strftime("%d %B %Y, %H:%M")

    note = (
        f"COMMUNITY HEALTH WORKER — REFERRAL NOTE\n"
        f"{'─' * 48}\n"
        f"Date & Time     : {timestamp}\n"
        f"Referred by     : Community Health Worker (CHW)\n"
        f"Tool            : Community Helper Guidebook (AI-assisted)\n"
        f"{'─' * 48}\n"
        f"CHW OBSERVATIONS:\n{case_summary}\n"
        f"{'─' * 48}\n"
        f"NOTE TO RECEIVING DOCTOR:\n"
        f"The above was captured by a community health worker using\n"
        f"an AI-assisted guidebook. This is NOT a clinical assessment.\n"
        f"Please conduct a full clinical evaluation.\n"
        f"{'─' * 48}\n"
    )

    return {
        "referral_note" : note,
        "timestamp"     : timestamp,
    }


# Tool registry 
TOOLS = {
    "get_first_aid_steps"     : get_first_aid_steps,
    "check_urgency"           : check_urgency,
    "lookup_nearest_facility" : lookup_nearest_facility,
    "generate_referral_note"  : generate_referral_note,
}

print("All 4 agentic tools ready:")
print("   1. get_first_aid_steps(situation)")
print("   2. check_urgency(symptoms)")
print("   3. lookup_nearest_facility(region)")
print("   4. generate_referral_note(case_summary)")

All 4 agentic tools ready:
   1. get_first_aid_steps(situation)
   2. check_urgency(symptoms)
   3. lookup_nearest_facility(region)
   4. generate_referral_note(case_summary)


In [46]:
# Module F — MedGemma Structured Agentic Pipeline

AGENT_SYSTEM_PROMPT = """You are a COMMUNITY HEALTH WORKER GUIDEBOOK ASSISTANT for low-resource Indian healthcare.

YOUR ROLE:
- Assist community health workers (CHWs), ASHA workers, NGO volunteers, and first-contact helpers.
- You do NOT speak to patients directly.
- You do NOT diagnose, prescribe medications, or recommend clinical treatments.
- You help helpers know what to say, what first aid to give, and when to escalate.
- Your guidance is grounded in Indian MoHFW and NHM regulations.

YOU WILL BE GIVEN:
- The helper's case report (translated to English)
- Indian healthcare regulatory context (from MoHFW/NHM documents)
- Pre-executed tool results: first aid steps, urgency level, nearest facility, referral note
- Optional: medical image context

YOUR ONLY JOB:
Use ALL the provided information to write a complete Helper Guidance Pack.
Do NOT call any tools. Do NOT add information not provided to you.
Do NOT diagnose. Do NOT suggest medications.

OUTPUT FORMAT — use this exactly:
══════════════════════════════════════════════
HELPER GUIDANCE PACK
══════════════════════════════════════════════

WHAT TO SAY TO THE PATIENT OR FAMILY:
[2-3 calm, simple, spoken-friendly sentences. No jargon. Helper reads these aloud.]

FIRST AID — DO THIS RIGHT NOW:
[Use the first aid steps provided. Keep them simple and spoken-friendly.]

WHAT NOT TO DO:
[Use the do-not steps provided. Include harmful home remedies to avoid.]

WHAT TO DO NEXT:
[Use the facility name and emergency number provided. Be specific.]

URGENCY:
[Use the urgency phrase provided word for word. Do not change it.]

══════════════════════════════════════════════

STRICT RULES:
1. NEVER diagnose any condition.
2. NEVER suggest medications, tablets, or injections.
3. NEVER provide clinical treatment steps.
4. NEVER change the urgency phrase — use it exactly as given.
5. ALWAYS name the specific facility provided in WHAT TO DO NEXT.
6. Use SIMPLE, SPOKEN language throughout.
7. Every section must be present and complete."""


def call_medgemma(
    messages       : list[dict],
    max_new_tokens : int   = 2048,
    temperature    : float = 0.3,
) -> str:
    """Helper to call MedGemma and decode only new tokens."""
    
    # Apply chat template and get input ids as tensor
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize              = True,
        add_generation_prompt = True,
        return_tensors        = "pt",
    )
    
    # Ensure we have a plain tensor not a BatchEncoding
    if hasattr(input_ids, "input_ids"):
        input_ids = input_ids.input_ids
    
    input_ids = input_ids.to(DEVICE)

    # Truncate if too long for GPU memory (keep last 3000 tokens)
    if input_ids.shape[-1] > 3000:
        input_ids = input_ids[:, -3000:]
        print(f"  Prompt truncated to 3000 tokens to fit GPU memory.")

    with torch.no_grad():
        output_ids = medgemma.generate(
            input_ids,
            max_new_tokens = max_new_tokens,
            temperature    = temperature,
            do_sample      = (temperature > 0),
            pad_token_id   = tokenizer.eos_token_id,
        )

    new_tokens = output_ids[0][input_ids.shape[-1]:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()


def run_react_agent(
    transcript    : str,
    rag_context   : str,
    image_context : dict | None = None,
    helper_notes  : str         = "",
    region        : str         = "India",
    max_steps     : int         = 4,   # kept for compatibility, not used in pre-execution
) -> tuple[str, list[dict]]:
    """
    Structured agentic pipeline:
    1. Pre-executes all 4 tools with case data (guaranteed, no loops)
    2. Passes all tool results + RAG context to MedGemma in one synthesis call
    3. MedGemma writes the complete Helper Guidance Pack

    Why pre-execution over free tool selection:
    - Eliminates infinite loops (tool called repeatedly with no progress)
    - Guarantees all sections are populated with real tool data
    - Prevents hallucinated urgency or first aid values
    - Faster — 1 synthesis call instead of 6-10 loop iterations
    - More reliable for a safety-critical healthcare application
    """
    reasoning_trace = []

    # ── STEP 1: Pre-execute all 4 tools ──────────────────────────
    print("   ↳ Pre-executing all 4 tools...")
    tool_results = {}

    # Tool 1: First aid steps
    try:
        tool_results["first_aid"] = get_first_aid_steps(transcript)
        print("   ✓ get_first_aid_steps")
    except Exception as e:
        print(f" get_first_aid_steps failed: {e}")
        tool_results["first_aid"] = {
            "first_aid": (
                "DO THIS NOW:\n"
                "1. Keep the person calm and still.\n"
                "2. Move them to a safe, shaded area.\n"
                "3. Loosen any tight clothing.\n"
                "4. Stay with them until help arrives.\n\n"
                "DO NOT DO:\n"
                "1. Do not give any medication.\n"
                "2. Do not leave them alone."
            )
        }

    reasoning_trace.append({
        "step"       : 1,
        "agent_output": "Tool called: get_first_aid_steps",
        "tool_called": "get_first_aid_steps",
        "tool_result": tool_results["first_aid"],
    })

    # Tool 2: Urgency check
    try:
        tool_results["urgency"] = check_urgency(transcript + " " + helper_notes)
        print("   ✓ check_urgency")
    except Exception as e:
        print(f" check_urgency failed: {e}")
        tool_results["urgency"] = {
            "urgency_phrase": (
                "This person should be seen by a health worker as soon as possible."
            )
        }

    reasoning_trace.append({
        "step"       : 2,
        "agent_output": "Tool called: check_urgency",
        "tool_called": "check_urgency",
        "tool_result": tool_results["urgency"],
    })

    # Tool 3: Nearest facility
    try:
        tool_results["facility"] = lookup_nearest_facility(region)
        print("   ✓ lookup_nearest_facility")
    except Exception as e:
        print(f" lookup_nearest_facility failed: {e}")
        tool_results["facility"] = {
            "facilities": ["Nearest Primary Health Centre (PHC)", "Community Health Centre (CHC)"],
            "emergency" : "108 — free ambulance, 24/7",
            "scheme_reminder": "Check Ayushman Bharat PM-JAY eligibility.",
        }

    reasoning_trace.append({
        "step"       : 3,
        "agent_output": "Tool called: lookup_nearest_facility",
        "tool_called": "lookup_nearest_facility",
        "tool_result": tool_results["facility"],
    })

    # Tool 4: Referral note
    try:
        tool_results["referral"] = generate_referral_note(
            f"{transcript}\nHelper notes: {helper_notes}\nRegion: {region}"
        )
        print("   ✓ generate_referral_note")
    except Exception as e:
        print(f" generate_referral_note failed: {e}")
        tool_results["referral"] = {
            "referral_note": f"CHW observation: {transcript}\nHelper notes: {helper_notes}"
        }

    reasoning_trace.append({
        "step"       : 4,
        "agent_output": "Tool called: generate_referral_note",
        "tool_called": "generate_referral_note",
        "tool_result": tool_results["referral"],
    })

    # ── STEP 2: Build image context ───────────────────────────────
    img_text = ""
    if image_context:
        img_text = f"\n\nIMAGE CONTEXT (SigLIP):\n{image_context['context_for_llm']}"

    notes_text = ""
    if helper_notes.strip():
        notes_text = f"\n\nHELPER ADDITIONAL OBSERVATIONS:\n{helper_notes.strip()}"

    # ── STEP 3: Build synthesis prompt with all tool results ──────
    synthesis_message = (
      
       f"CASE: \"{transcript}\"\n"
       f"Region: {region}\n"
       f"Helper notes: {helper_notes}\n\n"
       f"REGULATORY CONTEXT:\n{rag_context[:800]}\n\n"
       f"FIRST AID STEPS:\n{tool_results['first_aid']['first_aid'][:400]}\n\n"
       f"URGENCY: {tool_results['urgency']['urgency_phrase']}\n\n"
       f"NEAREST FACILITY: {', '.join(tool_results['facility']['facilities'][:2])}\n"
       f"EMERGENCY: {tool_results['facility']['emergency']}\n\n"
       f"Write the complete Helper Guidance Pack with all 5 sections now."
       
    )

    messages = [
        {"role": "system", "content": AGENT_SYSTEM_PROMPT},
        {"role": "user",   "content": synthesis_message},
    ]

    # ── STEP 4: Single MedGemma synthesis call ────────────────────
    print("   ↳ MedGemma synthesising guidance pack...")
    final_answer = call_medgemma(messages, max_new_tokens=2048, temperature=0.3)

    # Strip Final Answer: prefix if present
    if "Final Answer:" in final_answer:
        final_answer = final_answer.split("Final Answer:")[-1].strip()

    # ── STEP 5: If output looks truncated, retry once ─────────────
    if len(final_answer.split()) < 80:
        print(" Output looks short — retrying with explicit section request...")
        messages.append({"role": "assistant", "content": final_answer})
        messages.append({
            "role"    : "user",
            "content" : (
                "Your response was incomplete. Write the FULL Helper Guidance Pack "
                "with ALL 5 sections:\n"
                "1. WHAT TO SAY TO THE PATIENT OR FAMILY\n"
                "2. FIRST AID — DO THIS RIGHT NOW\n"
                "3. WHAT NOT TO DO\n"
                "4. WHAT TO DO NEXT\n"
                "5. URGENCY\n"
                "Use the tool results provided. All sections must be complete."
            ),
        })
        final_answer = call_medgemma(messages, max_new_tokens=2048, temperature=0.3)

    reasoning_trace.append({
        "step"        : 5,
        "agent_output": final_answer,
    })

    print(f"   ✓ Guidance pack generated ({len(final_answer.split())} words).")
    return final_answer, reasoning_trace


print(" MedGemma structured agentic pipeline ready.")
print("   Tools: get_first_aid_steps → check_urgency → lookup_nearest_facility → generate_referral_note → synthesis")

 MedGemma structured agentic pipeline ready.
   Tools: get_first_aid_steps → check_urgency → lookup_nearest_facility → generate_referral_note → synthesis


In [23]:
# Module G — Safety Guardrails

FORBIDDEN_PATTERNS = [
    "you have",
    "the patient has",
    "this is a case of",
    "this looks like",
    "take this medicine",
    "take paracetamol",
    "take ibuprofen",
    "give them medicine",
    "apply this cream",
    "administer",
    "prescribe",
    "diagnos",
    "it will get better",
    "nothing to worry",
    "don't worry it's fine",
]

IMMEDIATE_ESCALATION_KEYWORDS = [
    "chest pain", "difficulty breathing", "not breathing",
    "unconscious", "severe bleeding", "seizure", "fits",
    "convulsion", "snake bite", "poisoning", "stroke",
    "paralysis", "newborn not breathing", "premature birth",
    "not responding",
]


def apply_guardrails(
    guidance   : str,
    transcript : str,
) -> tuple[str, list[str], bool]:
    """
    Post-generation safety filter.
    Scans for forbidden phrases and immediate escalation triggers.
    Returns (guidance, warnings, requires_immediate_escalation).
    """
    warnings_list      = []
    requires_immediate = False

    for pattern in FORBIDDEN_PATTERNS:
        if pattern.lower() in guidance.lower():
            warnings_list.append(
                f" Review needed — phrase detected: '{pattern}'"
            )

    for kw in IMMEDIATE_ESCALATION_KEYWORDS:
        if kw in transcript.lower():
            requires_immediate = True
            break

    if requires_immediate:
        guidance += (
            "\n\nIMMEDIATE ACTION REQUIRED\n"
            "Signs of a serious emergency have been detected.\n"
            "Call 108 immediately — this is a free national ambulance service.\n"
            "Do not wait. Take the person to the nearest hospital now."
        )

    return guidance, warnings_list, requires_immediate


print("Safety guardrails ready.")

Safety guardrails ready.


In [53]:
# Module H — Full Pipeline Orchestrator

def run_full_pipeline(
    audio_path      : str | None         = None,
    text_input      : str                = "",
    confirmed_text  : str                = "",
    image           : Image.Image | None = None,
    helper_notes    : str                = "",
    input_language  : str                = "Hindi",
    output_language : str                = "Hindi",
    region          : str                = "India",
    verbose         : bool               = True,
) -> dict:
    """
    End-to-end pipeline:
    Audio/Text → STT → Confirmation → Translation →
    RAG Retrieval → Image Encoding → MedGemma ReAct Agent →
    Guardrails → Translation → Helper Guidance Pack
    """
    result     = {}
    input_mode = "audio" if audio_path else "text"
    result["input_mode"] = input_mode

    # ── STEP 1: Speech-to-Text ────────────────────────────────────
    if input_mode == "audio":
        if verbose: print("[1/8] Transcribing audio with Whisper...")
        lang_code                   = SUPPORTED_LANGUAGES.get(input_language, "hi")
        transcript, detected_lang   = transcribe_audio(audio_path, lang_code)
        result["transcript_raw"]    = transcript
        result["detected_language"] = detected_lang
        if verbose: print(f"   ✓ Transcript: {transcript[:80]}...")
    else:
        transcript                  = text_input.strip()
        result["transcript_raw"]    = transcript
        result["detected_language"] = SUPPORTED_LANGUAGES.get(input_language, "hi")
        if verbose: print(" [1/8] Text input — STT skipped.")

    # ── STEP 2: Confirmation Layer ────────────────────────────────
    if verbose: print("[2/8] Confirmation layer...")
    confirmation        = prepare_confirmation(transcript, input_mode)
    final_transcript    = confirmed_text.strip() if confirmed_text.strip() else transcript
    result["transcript_confirmed"] = final_transcript
    result["confirmation"]         = confirmation

    # ── STEP 3: Translate to English ──────────────────────────────
    if verbose: print(" [3/8] Translating to English (IndicTrans2)...")
    english_text, trans_src         = translate_to_english(final_transcript, input_language)
    result["transcript_english"]    = english_text
    result["translation_source"]    = trans_src
    if verbose: print(f"   ✓ {english_text[:80]}...")

    # ── STEP 4: RAG Retrieval ─────────────────────────────────────
    if verbose: print("[4/8] Retrieving Indian healthcare regulations (RAG)...")
    rag_query   = f"{english_text} {helper_notes} region: {region}"
    rag_context = retrieve_context(rag_query, rag_index, rag_doc_store, top_k=4)
    result["rag_context"] = rag_context
    if verbose: print(f"   ✓ Retrieved {len(rag_context.split())} words of regulatory context.")

    # ── STEP 5: Image Encoding ────────────────────────────────────
    image_context = None
    if image is not None:
        if verbose: print(" [5/8] Encoding medical image (SigLIP)...")
        image_context = encode_image(image)
        result["image_context"] = image_context
        if verbose: print(f"   ✓ Detected: {image_context['image_type']} ({image_context['confidence']:.0%})")
    else:
        if verbose: print(" [5/8] No image — skipped.")
        result["image_context"] = None

    # ── STEP 6: MedGemma ReAct Agent ─────────────────────────────
    if verbose: print("[6/8] Running MedGemma ReAct agent (RAG-grounded)...")
    guidance_en, reasoning_trace = run_react_agent(
        transcript    = english_text,
        rag_context   = rag_context,
        image_context = image_context,
        helper_notes  = helper_notes,
        region        = region,
    )
    result["guidance_english"]  = guidance_en
    result["reasoning_trace"]   = reasoning_trace
    if verbose: print(f"   ✓ Agent completed in {len(reasoning_trace)} reasoning steps.")

    # ── STEP 7: Safety Guardrails ─────────────────────────────────
    if verbose: print(" [7/8] Applying safety guardrails...")
    safe_guidance, warnings, immediate = apply_guardrails(guidance_en, english_text)
    result["safety_warnings"]          = warnings
    result["requires_immediate"]       = immediate
    if immediate and verbose:
        print("   IMMEDIATE ESCALATION TRIGGERED.")

    # ── STEP 8: Translate Output to Local Language ────────────────
    if verbose: print(f" [8/8] Translating output to {output_language}...")

    final_pack_en = (
        safe_guidance
        + DISCLAIMER
    )

    final_pack_local, _ = translate_from_english(final_pack_en, output_language)

    result["guidance_pack_english"] = final_pack_en
    result["guidance_pack_local"]   = final_pack_local
    result["output_language"]       = output_language

    if verbose:
       print("\n" + final_pack_local)
       for w in warnings:
           print(w)

    return result


print("Full pipeline orchestrator ready.")

Full pipeline orchestrator ready.


In [54]:
# Synthetic Demo Scenarios

DEMO_SCENARIOS = [
    {
        "id"             : 1,
        "title"          : "Scenario 1 — Child with Prolonged Fever (Hindi output)",
        "input_language" : "Hindi",
        "output_language": "Hindi",
        "text_input"     : (
            "Baccha teen din se bukhaar mein hai. Woh kuch kha nahi raha. "
            "Raat ko kaafi zyada bukhaar badh jaata hai. Woh paanch saal ka hai."
        ),
        "helper_notes"   : "Child looks very weak. Family is 40km from nearest hospital. No vehicle available.",
        "region"         : "Rajasthan",
        "image"          : None,
    },
    {
        "id"             : 2,
        "title"          : "Scenario 2 — Burn Injury with Image (Hindi output)",
        "input_language" : "English",
        "output_language": "Hindi",
        "text_input"     : (
            "The woman's hand got burned while cooking. There is redness and "
            "blistering on the palm. She is in a lot of pain. "
            "It happened two hours ago. Someone applied mustard oil — we told them to stop."
        ),
        "helper_notes"   : "No doctor in the village. Nearest PHC is 25km away. Family has Ayushman Bharat card.",
        "region"         : "Maharashtra",
        "image"          : "placeholder_burn",
    },
    {
        "id"             : 3,
        "title"          : "Scenario 3 — Chest Pain & Breathlessness (Immediate escalation, English output)",
        "input_language" : "English",
        "output_language": "English",
        "text_input"     : (
            "An elderly man around 65 years old is having chest pain and "
            "difficulty breathing. He is sweating heavily and his left arm is hurting. "
            "His lips look pale. The pain started about 30 minutes ago."
        ),
        "helper_notes"   : "108 ambulance called but 45 minutes away. No oxygen available. Son is present.",
        "region"         : "Uttar Pradesh",
        "image"          : None,
    },
]


def run_demo(scenario: dict) -> dict:
    print(f"\n{'=' * 70}")
    print(f"{scenario['title']}")
    print(f"{'=' * 70}")

    image = None
    if scenario["image"] == "placeholder_burn":
        image = Image.new("RGB", (400, 300), color=(255, 200, 180))
        draw  = ImageDraw.Draw(image)
        draw.text(
            (200, 150),
            "Burn injury image (demo placeholder)",
            fill   = (180, 60, 30),
            anchor = "mm",
        )

    return run_full_pipeline(
        text_input      = scenario["text_input"],
        image           = image,
        helper_notes    = scenario["helper_notes"],
        input_language  = scenario["input_language"],
        output_language = scenario["output_language"],
        region          = scenario["region"],
        verbose         = True,
    )


print(" Running demo scenarios...\n")
all_results = []
for scenario in DEMO_SCENARIOS:
    res = run_demo(scenario)
    all_results.append(res)
    print()

print("All demo scenarios completed.")

 Running demo scenarios...


Scenario 1 — Child with Prolonged Fever (Hindi output)
 [1/8] Text input — STT skipped.
[2/8] Confirmation layer...
 [3/8] Translating to English (IndicTrans2)...
   ✓ The child has been suffering from fever since day one. He was not eating. There ...
[4/8] Retrieving Indian healthcare regulations (RAG)...
   ✓ Retrieved 1565 words of regulatory context.
 [5/8] No image — skipped.
[6/8] Running MedGemma ReAct agent (RAG-grounded)...
   ↳ Pre-executing all 4 tools...
   ✓ get_first_aid_steps
   ✓ check_urgency
   ✓ lookup_nearest_facility
   ✓ generate_referral_note
   ↳ MedGemma synthesising guidance pack...
   ✓ Guidance pack generated (126 words).
   ✓ Agent completed in 5 reasoning steps.
 [7/8] Applying safety guardrails...
 [8/8] Translating output to Hindi...

═══════════════════════ ═══════════════════════
सहायक मार्गदर्शन पैक
═══════════════════════ ═══════════════════════

मरीज़ या परिवार से क्या कहें:
नमस्ते, मैं मदद के लिए यहां हूं। मैं समझता हूं

In [55]:
# Gradio Interactive UI

import gradio as gr

LANG_LIST = list(SUPPORTED_LANGUAGES.keys())


def gradio_transcribe(audio_path: str, input_language: str) -> tuple[str, str]:
    """Step 1: Transcribe audio and return for helper confirmation."""
    if not audio_path:
        return "", "No audio provided."
    try:
        lang_code            = SUPPORTED_LANGUAGES.get(input_language, "hi")
        transcript, detected = transcribe_audio(audio_path, lang_code)
        status = (
            f"Transcription complete (detected: {detected}).\n"
            "Please review the text below and correct any errors before proceeding."
        )
        return transcript, status
    except Exception as e:
        return "", f" Transcription error: {str(e)}"


def sync_output_language(input_lang: str) -> gr.update:
    """Auto-sync output language dropdown to match input language."""
    return gr.update(value=input_lang)


def gradio_generate(
    audio_path      : str | None,
    confirmed_text  : str,
    text_direct     : str,
    image_path      : str | None,
    helper_notes    : str,
    input_language  : str,
    output_language : str,
    region          : str,
) -> tuple[str, str, str, str]:
    """Full pipeline handler. Returns (input_summary, image_analysis, reasoning_trace, guidance_pack)."""
    try:
        # ── Resolve text input ─────────────────────────────────────
        if audio_path and confirmed_text.strip():
            text_for_pipe = confirmed_text.strip()
        elif text_direct.strip():
            text_for_pipe = text_direct.strip()
        else:
            return "Please provide audio or type notes before generating.", "", "", ""

        # ── Handle image ──────────────────────────────────────────
        image = None
        if image_path:
            image = Image.open(image_path).convert("RGB")

        # ── Run pipeline ──────────────────────────────────────────
        result = run_full_pipeline(
            text_input      = text_for_pipe,
            confirmed_text  = text_for_pipe,
            image           = image,
            helper_notes    = helper_notes,
            input_language  = input_language,
            output_language = output_language,
            region          = region,
            verbose         = False,
        )

        # ── Input summary ─────────────────────────────────────────
        input_summary = (
            f"Input mode     : {'Audio (transcribed)' if audio_path else 'Text'}\n"
            f"Language       : {input_language} → {output_language}\n"
            f"Region         : {region}\n"
            f"RAG docs used  : {len(result['rag_context'].split())} words of Indian regulatory context\n\n"
            f"Text used      :\n{result['transcript_confirmed']}"
        )

        # ── Image analysis ────────────────────────────────────────
        img_info = "No image uploaded."
        if result.get("image_context"):
            ctx      = result["image_context"]
            top_alts = sorted(ctx["all_labels"].items(), key=lambda x: -x[1])[:4]
            img_info = (
                f"Detected type  : {ctx['image_type']}\n"
                f"Confidence     : {ctx['confidence']:.0%}\n\n"
                f"Top alternatives:\n"
                + "\n".join(f"  • {k}: {v:.0%}" for k, v in top_alts)
            )

        # ── Reasoning trace ───────────────────────────────────────
        trace = "MedGemma Structured Agent — Tool Execution Trace:\n" + "─" * 40 + "\n"
        for step in result.get("reasoning_trace", []):
            trace += f"\nStep {step['step']}:\n"
            if step.get("tool_called"):
                trace += f"  → Tool called : {step['tool_called']}\n"
                if step.get("tool_result"):
                    result_preview = str(step['tool_result'])[:200]
                    trace += f"  → Result      : {result_preview}...\n"
            else:
                trace += f"{step['agent_output'][:300]}\n"

        # ── Final guidance ────────────────────────────────────────
        guidance = result["guidance_pack_local"]
        if result.get("safety_warnings"):
            guidance += "\n\n" + "\n".join(result["safety_warnings"])

        return input_summary, img_info, trace, guidance

    except Exception as e:
        import traceback
        err = f" Error: {str(e)}\n{traceback.format_exc()}"
        return err, "", "", ""


# ── Build Gradio UI ───────────────────────────────────────────────

with gr.Blocks(
    title = " Community Helper Guidebook",
    theme = gr.themes.Soft(),
) as demo:

    gr.Markdown("""
    #  Community Helper Guidebook
    ### Offline, Multimodal Healthcare Assistant for Low-Resource India
    *Grounded in MoHFW / NHM / National Health Policy 2017*
    ---
    > **Assistive guidance only.** Does NOT replace clinical advice.
    > Always escalate to a qualified healthcare professional. Emergency: **108**

    **For:** ASHA Workers · Community Health Workers · NGO Volunteers · First-contact helpers
    **Not for:** Patients directly · Clinical diagnosis · Treatment decisions
    ---
    """)

    # ── Language + Region ─────────────────────────────────────────
    with gr.Row():
        input_lang = gr.Dropdown(
            choices = LANG_LIST,
            value   = "Hindi",
            label   = "Helper's Language",
        )
        output_lang = gr.Dropdown(
            choices = LANG_LIST,
            value   = "Hindi",
            label   = "Output Language (defaults to input language)",
        )
        region_box = gr.Textbox(
            value       = "India",
            label       = "Region / State",
            placeholder = "e.g. Maharashtra, Rajasthan, Bihar",
        )

    # Sync output language when input changes
    input_lang.change(
        fn      = sync_output_language,
        inputs  = input_lang,
        outputs = output_lang,
    )

    # ── Input tabs ────────────────────────────────────────────────
    with gr.Tabs():

        with gr.TabItem(" Voice Note"):
            gr.Markdown(
                "Upload a voice note in any format, or **record directly** in the browser."
            )
            audio_input = gr.Audio(
                label   = "Voice Note",
                sources = ["upload", "microphone"],
                type    = "filepath",
            )
            transcribe_btn    = gr.Button("Transcribe Audio", variant="secondary")
            transcribe_status = gr.Textbox(
                label       = "Transcription Status",
                interactive = False,
                lines       = 2,
            )
            confirmed_text = gr.Textbox(
                label       = "Review & Correct Transcription",
                lines       = 4,
                placeholder = "Transcription appears here after clicking Transcribe. Correct any errors.",
            )
            transcribe_btn.click(
                fn      = gradio_transcribe,
                inputs  = [audio_input, input_lang],
                outputs = [confirmed_text, transcribe_status],
            )

        with gr.TabItem("Type Notes"):
            gr.Markdown("Type the helper's observation directly.")
            text_direct = gr.Textbox(
                label       = "Helper's Notes",
                lines       = 5,
                placeholder = "e.g. Baccha teen din se bukhaar mein hai...",
            )

        with gr.TabItem("Medical Image"):
            gr.Markdown("Upload a photo if available (optional).")
            image_input = gr.Image(
                label  = "Medical Image (wound, rash, X-ray, prescription, etc.)",
                type   = "filepath",
            )

    # ── Helper notes ──────────────────────────────────────────────
    helper_notes_box = gr.Textbox(
        label       = "Helper's Additional Observations",
        lines       = 2,
        placeholder = (
            "e.g. Child looks very weak. Family 40km from hospital. "
            "No vehicle available. Family has Ayushman Bharat card."
        ),
    )

    # ── Generate ──────────────────────────────────────────────────
    generate_btn = gr.Button(
        "Generate Helper Guidance Pack",
        variant = "primary",
        size    = "lg",
    )

    # ── Outputs ───────────────────────────────────────────────────
    gr.Markdown("---\n### Results")

    with gr.Row():
        with gr.Column(scale=1):
            input_summary_out = gr.Textbox(
                label       = "Input Summary",
                lines       = 8,
                interactive = False,
            )
            image_out = gr.Textbox(
                label       = "Image Analysis (SigLIP)",
                lines       = 8,
                interactive = False,
            )
            trace_out = gr.Textbox(
                label       = "Agent Tool Execution Trace",
                lines       = 12,
                interactive = False,
            )

        with gr.Column(scale=1):
            guidance_out = gr.Textbox(
                label       = "Helper Guidance Pack",
                lines       = 30,
                interactive = False,
            )

    generate_btn.click(
        fn      = gradio_generate,
        inputs  = [
            audio_input, confirmed_text, text_direct,
            image_input, helper_notes_box,
            input_lang, output_lang, region_box,
        ],
        outputs = [input_summary_out, image_out, trace_out, guidance_out],
    )

    # ── Demo examples ─────────────────────────────────────────────
    gr.Markdown("---\n### Quick Demo Examples (click to load)")
    gr.Examples(
        examples = [
            [
                None, "",
                "Baccha teen din se bukhaar mein hai. Woh kuch kha nahi raha.",
                None,
                "Child looks very weak. 40km from hospital. No vehicle.",
                "Hindi", "Hindi", "Rajasthan",
            ],
            [
                None, "",
                "The woman's hand got burned while cooking. Redness and blistering on the palm.",
                None,
                "Mustard oil was applied. No doctor nearby. Family has Ayushman Bharat card.",
                "English", "Hindi", "Maharashtra",
            ],
            [
                None, "",
                "Elderly man having chest pain and difficulty breathing. Left arm hurting. Sweating heavily.",
                None,
                "108 called, 45 minutes away. No oxygen available.",
                "English", "English", "Uttar Pradesh",
            ],
        ],
        inputs = [
            audio_input, confirmed_text, text_direct,
            image_input, helper_notes_box,
            input_lang, output_lang, region_box,
        ],
    )


In [56]:
# ── Launch ────────────────────────────────────────────────────────
print("\nLaunching Gradio...")
demo.launch(share=True, debug=False)


Launching Gradio...
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://981a1d49861e7ac1a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


   ↳ Pre-executing all 4 tools...
   ✓ get_first_aid_steps
   ✓ check_urgency
   ✓ lookup_nearest_facility
   ✓ generate_referral_note
   ↳ MedGemma synthesising guidance pack...
   ✓ Guidance pack generated (176 words).
